In [2]:
import numpy as np
import jax
import jax.numpy as jnp
import scipy.stats as ss
import tensorly as tl
tl.set_backend('jax')
# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

In [3]:
sample = np.genfromtxt('references/sample.csv', delimiter=',')

In [4]:
# S = jnp.array([
#             [1.00, 1.09, 1.08, 1.34],
#             [1.00, 1.16, 1.26, 1.54],
#             [1.00, 1.22, 1.07, 1.03],
#             [1.00, 0.93, 0.97, 0.92],
#             [1.00, 1.11, 1.56, 1.52],
#             [1.00, 0.76, 0.77, 0.90],
#             [1.00, 0.92, 0.84, 1.01],
#             [1.00, 0.88, 1.22, 1.34]])
S = jnp.transpose(sample)

In [12]:
# check distribution
print(np.mean(S[:-1]))
print(np.var(S[:-1]))
# 1.05

1.049455556891717
0.034085816282478955


In [6]:
N = S.shape[1]          # number of time steps
r = 0.05       # interest rate
K = 1          # strike 
T = 1          # Maturity

dt = T/(N-1)          # time interval
df = jnp.exp(-r * dt)  # discount factor per time interval

In [7]:
H = np.maximum(K - S, 0)           # intrinsic values for put option

V = np.zeros_like(H)               # value matrix
V[:,-1] = H[:,-1]

# Valuation by LS Method
for t in range(N-2, 0, -1):

    good_paths = H[:,t] > 0        # paths where the intrinsic value is positive 
                                   # the regression is performed only on these paths 
    
    rg = jnp.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    # polynomial regression
    C = jnp.polyval( rg, S[good_paths,t] )                             # evaluation of regression  
    
    exercise = np.zeros( len(good_paths), dtype=bool)    # initialize
    exercise[good_paths] = H[good_paths,t] > C           # paths where it is optimal to exercise
    
    V[exercise,t] = H[exercise,t]                        # set V equal to H where it is optimal to exercise 
    V[exercise,t+1:] = 0                                 # set future cash flows, for that path, equal to zero  
    discount_path = (V[:,t] == 0)                        # paths where we didn't exercise 
    V[discount_path,t] = V[discount_path,t+1] * df       # set V[t] in continuation region
    
V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
print("Example price= ", V0)

Example price=  0.07437147205824303


In [8]:
print(H.round(2))

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.43 0.43 0.44]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.1  0.1  0.1 ]]


In [9]:
# 0.0797